In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [6]:
Invitations=pd.read_csv(r'C:\Users\Andrea\Desktop\My_LinkedIn\Basic_Information_Files\Invitations.csv')
Invitations.head()

,From,To,Sent At,Message,Direction
0,Marcela Huerfano,Haldo Spontón,"12/12/20, 10:16 AM",NaN,OUTGOING
1,Marcela Huerfano,Marta Sylvia del Rio Guerra,"12/11/20, 3:48 PM",NaN,OUTGOING
2,Marcela Huerfano,Nicolás E. Ozimica,"12/11/20, 12:43 PM",NaN,OUTGOING
3,Marcela Huerfano,Jose Ricardo Zapata Gonzalez,"12/11/20, 5:52 AM",NaN,OUTGOING
4,Marcela Huerfano,Nicolas Martinez Baquero,"12/9/20, 10:19 AM",NaN,OUTGOING


In [9]:
Invitations.dtypes

From         object
To           object
Sent At      object
Message      object
Direction    object
dtype: object